<style>
  .blue-box {
    background-color: #f0f0f0; /* grey background */
    padding: 10px;
    border: 1px solid #ccc;
    text-align: center;
  }
  .blue-text {
    color: #0000ff; /* blue text */
    font-weight: bold;
  }
</style>

<div class="blue-box">
  <h2 class="blue-text">Forecasting VN1 Data at Weekly Time Granularity with TimesFM</h2>
</div>



### Installing TimesFM and additional Libraries

In [ ]:
!pip install jax paxml sympy -q

In [ ]:
!pip install openpyxl -q

In [ ]:
!pip install timesfm -q


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import timesfm

TimesFM v1.2.0. See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded Jax TimesFM.


c:\Users\317379\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded PyTorch TimesFM.


#### Reading the data

In [ ]:
data = pd.read_excel("VN1_train.xlsx")
data.head()

,unique_id,ds,y,week_of_year_sin,week_of_year_cos,is_thanksgiving_week,is_christmas_week,is_holiday_week,trend,seasonal
0,C0_W100_P10705,2020-07-06,7,-0.176890,-0.984231,0.0,0.0,False,6.924783,0.823216
1,C0_W100_P10705,2020-07-13,7,-0.292057,-0.956401,0.0,0.0,False,6.902250,0.835461
2,C0_W100_P10705,2020-07-20,7,-0.403123,-0.915146,0.0,0.0,False,6.879824,0.866956
3,C0_W100_P10705,2020-07-27,7,-0.508531,-0.861044,0.0,0.0,False,6.857521,0.897566
4,C0_W100_P10705,2020-08-03,7,-0.606800,-0.794854,0.0,0.0,False,6.835363,0.927268


In [ ]:
# Convert to date to datetime format
data['ds'] = pd.to_datetime(data['ds'])

In [ ]:
#Defining train and test partitions

Y_test_df = pd.read_excel("VN1_test.xlsx")
df = data.copy()

### Model Building

In [ ]:
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="cpu",
          per_core_batch_size=32,
          horizon_len=13,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),
  )

c:\Users\317379\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Fetching 3 files: 100%|██████████| 3/3 [00:00<?, ?it/s]


In [ ]:
timesfm_df = tfm.forecast_on_df(
    inputs=df,
    freq='W',
    value_name='y',
    num_jobs=-1,
)

Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.


In [ ]:
#Forecast
timesfm_df


,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,C0_W100_P10705,2023-10-08,3.900283,2.544355,3.109720,3.325919,3.653064,3.900283,4.137675,4.432699,4.722989,5.273695
1,C0_W100_P10705,2023-10-15,4.060882,2.483986,3.170091,3.465308,3.883624,4.060882,4.311878,4.653979,5.041049,5.541929
2,C0_W100_P10705,2023-10-22,4.161891,2.568358,3.136958,3.611173,3.870723,4.161891,4.442762,4.720335,5.109875,5.760804
3,C0_W100_P10705,2023-10-29,4.198008,2.450525,3.240373,3.604379,3.883821,4.198008,4.424438,4.720629,5.098867,5.728715
4,C0_W100_P10705,2023-11-05,4.248225,2.563373,3.329753,3.669714,3.996807,4.248225,4.600279,4.930752,5.264300,5.773182
...,...,...,...,...,...,...,...,...,...,...,...,...
53269,C7_W303_P8795,2023-12-03,0.205902,-0.089790,-0.006844,0.043363,0.091200,0.205902,0.316194,0.466015,0.784196,1.521213
53270,C7_W303_P8795,2023-12-10,0.164305,-0.054381,0.006141,0.050390,0.116043,0.164305,0.307730,0.441513,0.777021,1.560258
53271,C7_W303_P8795,2023-12-17,0.204870,-0.036982,0.009575,0.089707,0.136536,0.204870,0.300042,0.467726,0.771042,1.552811
53272,C7_W303_P8795,2023-12-24,0.247441,-0.006052,0.027045,0.087347,0.176400,0.247441,0.354179,0.536357,0.850476,1.613036


In [28]:
timesfm_df.to_csv("timesfm_weekly_VN1.csv")